Task1_0724. Wine 데이터셋에 대하여 SVM 모델에 3개의 커널을 적용하여 학습 및 평가 결과를 출력하세요.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

wine = datasets.load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

kernels = ['linear', 'poly', 'rbf']

for kernel in kernels:
  svm = SVC(kernel=kernel, random_state=42)
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)

  print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.97
Accuracy: 0.97
Accuracy: 1.00


Task2_0724. wind dataset으로 랜덤포레스트를 적용하여 모델링 및 평가를 아래의 하이퍼 파라미터를 이용하여 수행한 후 최적의 하이퍼파라미터를 구하세요.

- 'n_estimators': [50, 100, 200],
- 'max_depth': [None, 10, 20],
- 'max_features': ['auto', 'sqrt', 'log2'],
- 'min_samples_split': [2, 5, 10],
- 'min_samples_leaf': [1, 2, 4]

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#clf = RandomForestClassifier(n_estimators=100, random_state=42) # 9649
#clf = RandomForestClassifier(n_estimators=200, random_state=42) # 9649
#clf = RandomForestClassifier(n_estimators=50, random_state=42) # 9649

#clf = RandomForestClassifier(max_depth=None, random_state=42) # 9649
#clf = RandomForestClassifier(max_depth=10, random_state=42) # 9649
#clf = RandomForestClassifier(max_depth=20, random_state=42) # 9649

#clf = RandomForestClassifier(max_features='auto', random_state=42) # warning
#clf = RandomForestClassifier(max_features='sqrt', random_state=42) # 9649
#clf = RandomForestClassifier(max_features='log2', random_state=42) # 9649

#clf = RandomForestClassifier(min_samples_split=2, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_split=5, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_split=10, random_state=42) # 9649

#clf = RandomForestClassifier(min_samples_leaf=1, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_leaf=2, random_state=42) # 9649
#clf = RandomForestClassifier(min_samples_leaf=4, random_state=42) # 9649

clf = RandomForestClassifier(n_estimators=50, max_depth=20, max_features='sqrt', min_samples_split=5, min_samples_leaf=2, random_state=42)


clf.fit(X_train, y_train)
pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, pred)
print(f"Accuracy : {accuracy:.4f}")

Accuracy : 0.9649
